# How to Interpolate?

In [ ]:
#user-defined
import ipynb.fs.full.helper as hp

#dataframes
import pandas as pd
import h5py
import rosbag_pandas

#math
import numpy as np
import math
pd.TimeSeries = pd.Series 

#interpolation scipy
import scipy
from scipy import signal
import scipy.interpolate as interpolate
from scipy import integrate

#gps
from geopy import distance
import geopandas as gpd
import utm
import datetime as dt

#plots
import pylab as plt
import matplotlib.pyplot as plt
import matplotlib.animation as animation  



### Using Rosbag Pandas to import Bag

In [2]:
def clean_df(df):
    df.columns=df.columns.str.replace(r"/", "_")
    df = df[df.columns.drop(list(df.filter(regex='covariance')))]
    df = df[df.columns.drop(list(df.filter(regex='frame_id')))]
    df = df[df.columns.drop(list(df.filter(regex='twist_twist_angular')))]
    df = df[df.columns.drop(list(df.filter(regex='seq')))]
    return df
def rosbag_to_pandas(filename, topics):  
    d = {name: pd.DataFrame() for name in topics}
    data = []
    for name,df in d.items():
        df = rosbag_pandas.bag_to_dataframe(filename, include=[name])
        df = clean_df(df)
        data.append(df)
    data=pd.concat(data)
    return data

In [5]:
run = 4
for i in range(0,6):
    set_number = i
    filename = '/home/ecc/data/BagsOctober/Run0'+str(run)+'/OctoberDesert1020_'+str(set_number)+'.bag'
    topics = ('/trisonica','/analog_output','/ublox_gps/fix', '/ublox_gps/fix_velocity','/imu/data')
    df= rosbag_to_pandas(filename, topics)
    df = df.reset_index()
    df = df.rename(columns={"index": "master_time"})
    df.to_hdf('/home/ecc/data/DesertDataOct/Run0'+str(run)+'/Run0'+str(run)+
              'Set0'+str(set_number)+'.h5', key='df', mode='w')
# df.keys()

### Load Data

In [46]:
data = []
run = 4
for i in range(0,6):
    set_number = i 
    f_name = '/home/ecc/data/DesertDataOct/Run0'+str(run)+'/Run0'+str(run)+'Set0'+str(set_number)+'.h5'    
    df = pd.read_hdf(f_name)
    data.append(df)
df=pd.concat(data)    

## Interpolation

### Getting the GPS and WindSensor Time

In [47]:
df["trisonica_time"] = df._trisonica_header_stamp_secs + (df._trisonica_header_stamp_nsecs*1e-9)
df["gps_time"] = df._ublox_gps_fix_header_stamp_secs+(df._ublox_gps_fix_header_stamp_nsecs*1e-9)
df["gps_vel_time"] = df._ublox_gps_fix_velocity_header_stamp_secs+(df._ublox_gps_fix_velocity_header_stamp_nsecs*1e-9)
df["imu_time"] = df._imu_data_header_stamp_secs +(df._imu_data_header_stamp_nsecs*1e-9)

### Separating ODOR, WIND, GPS in dataframes

In [48]:
df_odor = pd.DataFrame({'odor_time': df.master_time,
                        'odor': df._analog_output_data,})
df_wind = pd.DataFrame({'wind_time': df.trisonica_time,'trisonica_U': df._trisonica_westeast,
                        'trisonica_V': df._trisonica_northsouth,'trisonica_pressure': df._trisonica_pressure,
                       'trisonica_temperature': df._trisonica_temperature})
df_gps = pd.DataFrame({'gps_time': df.gps_time,'lat': df._ublox_gps_fix_latitude,
                       'lon': df._ublox_gps_fix_longitude})
df_gps_vel = pd.DataFrame({'gps_vel_time':df.gps_vel_time,
                        'gps_linear_x':df._ublox_gps_fix_velocity_twist_twist_linear_x,
                      'gps_linear_y':df._ublox_gps_fix_velocity_twist_twist_linear_y,
                      'gps_linear_z':df._ublox_gps_fix_velocity_twist_twist_linear_z})
df_imu = pd.DataFrame({'imu_time': df.imu_time,'imu_angular_x': df._imu_data_angular_velocity_x,
                     'imu_angular_y': df._imu_data_angular_velocity_y, 
                     'imu_angular_z':df._imu_data_angular_velocity_z,
                      'imu_linear_acc_x':df._imu_data_linear_acceleration_x,
                      'imu_linear_acc_y':df._imu_data_linear_acceleration_y,
                      'imu_linear_acc_z':df._imu_data_linear_acceleration_z})

#### Creating time column for odor by dropping NaNs

In [49]:
df_odor = df_odor[df_odor.odor.notna()]

### Combined time from three sensors and arrange based on time

In [50]:
dfn = pd.concat([df_odor, df_wind, df_gps, df_gps_vel, df_imu], sort=True)
combined_time = np.hstack([df_odor.odor_time, df.trisonica_time ,df.gps_time, df.gps_vel_time, df.imu_time])
dfn['time'] = combined_time
dfn = dfn.sort_values(by=['time'])

In [51]:
dfn.keys()

Index(['gps_linear_x', 'gps_linear_y', 'gps_linear_z', 'gps_time',
       'gps_vel_time', 'imu_angular_x', 'imu_angular_y', 'imu_angular_z',
       'imu_linear_acc_x', 'imu_linear_acc_y', 'imu_linear_acc_z', 'imu_time',
       'lat', 'lon', 'odor', 'odor_time', 'trisonica_U', 'trisonica_V',
       'trisonica_pressure', 'trisonica_temperature', 'wind_time', 'time'],
      dtype='object')

### Find Master Time 

In [52]:
dt_odor = np.median(np.diff(dfn.odor_time.dropna().values))
print('dt Odor: ', dt_odor)
dt_wind = np.median(np.diff(dfn.wind_time.dropna().values))
print('dt Wind: ', dt_wind)
dt_gps = np.median(np.diff(dfn.gps_time.dropna().values))
print('dt Gps: ', dt_gps)
dt_gps_vel = np.median(np.diff(dfn.gps_vel_time.dropna().values))
print('dt Gps Vel: ', dt_gps_vel)
dt_imu = np.median(np.diff(dfn.imu_time.dropna().values))
print('dt Imu: ', dt_imu)
first_time = np.max([dfn.odor_time.min(), dfn.wind_time.min(), dfn.gps_vel_time.min(),
                     dfn.gps_time.min(), dfn.imu_time.min()])
last_time = np.min([dfn.odor_time.max(), dfn.wind_time.max(), dfn.gps_time.max(),
                    dfn.gps_vel_time.max(),dfn.imu_time.max()])
# print(first_time,last_time)
master_time = np.arange(first_time, last_time, dt_odor)

dt Odor:  0.004996538162231445
dt Wind:  0.0249941349029541
dt Gps:  0.20000004768371582
dt Gps Vel:  0.20000004768371582
dt Imu:  0.003999948501586914


In [53]:
dfn.keys()

Index(['gps_linear_x', 'gps_linear_y', 'gps_linear_z', 'gps_time',
       'gps_vel_time', 'imu_angular_x', 'imu_angular_y', 'imu_angular_z',
       'imu_linear_acc_x', 'imu_linear_acc_y', 'imu_linear_acc_z', 'imu_time',
       'lat', 'lon', 'odor', 'odor_time', 'trisonica_U', 'trisonica_V',
       'trisonica_pressure', 'trisonica_temperature', 'wind_time', 'time'],
      dtype='object')

### Create Interpolation function

In [54]:
# create interpolation function and inteprolate sensor 1
df_odor_1= dfn[['odor_time', 'odor']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_odor_1.odor_time, df_odor_1.odor)
interp_odor = func_interp_sensor_1(master_time)

df_wind_1= dfn[['wind_time', 'trisonica_U']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_wind_1.wind_time, df_wind_1.trisonica_U)
interp_wind_U = func_interp_sensor_1(master_time)

df_wind_2= dfn[['wind_time', 'trisonica_V']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_wind_2.wind_time, df_wind_2.trisonica_V)
interp_wind_V = func_interp_sensor_1(master_time)

df_wind_3= dfn[['wind_time', 'trisonica_pressure']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_wind_3.wind_time, df_wind_3.trisonica_pressure)
interp_wind_p = func_interp_sensor_1(master_time)

df_wind_4= dfn[['wind_time', 'trisonica_temperature']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_wind_4.wind_time, df_wind_4.trisonica_temperature)
interp_wind_t = func_interp_sensor_1(master_time)


df_gps_lat= dfn[['gps_time', 'lat']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_gps_lat.gps_time, df_gps_lat.lat)
interp_gps_lat = func_interp_sensor_1(master_time)

df_gps_lon= dfn[['gps_time', 'lon']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_gps_lon.gps_time, df_gps_lon.lon)
interp_gps_lon = func_interp_sensor_1(master_time)

df_gps_x= dfn[['gps_vel_time', 'gps_linear_x']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_gps_x.gps_vel_time, df_gps_x.gps_linear_x)
interp_gps_x = func_interp_sensor_1(master_time)

df_gps_y= dfn[['gps_vel_time', 'gps_linear_y']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_gps_y.gps_vel_time, df_gps_y.gps_linear_y)
interp_gps_y = func_interp_sensor_1(master_time)

df_gps_z= dfn[['gps_vel_time', 'gps_linear_z']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_gps_z.gps_vel_time, df_gps_z.gps_linear_z)
interp_gps_z = func_interp_sensor_1(master_time)


df_imu_x= dfn[['imu_time', 'imu_angular_x']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_imu_x.imu_time, df_imu_x.imu_angular_x)
interp_imu_x = func_interp_sensor_1(master_time)

df_imu_y= dfn[['imu_time', 'imu_angular_y']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_imu_y.imu_time, df_imu_y.imu_angular_y)
interp_imu_y = func_interp_sensor_1(master_time)


df_imu_z= dfn[['imu_time', 'imu_angular_z']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_imu_z.imu_time, df_imu_z.imu_angular_z)
interp_imu_z = func_interp_sensor_1(master_time)


df_imu_linear_x= dfn[['imu_time', 'imu_linear_acc_x']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_imu_linear_x.imu_time, df_imu_linear_x.imu_linear_acc_x)
interp_imu_linear_acc_x = func_interp_sensor_1(master_time)

df_imu_linear_y= dfn[['imu_time', 'imu_linear_acc_y']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_imu_linear_y.imu_time, df_imu_linear_y.imu_linear_acc_y)
interp_imu_linear_acc_y = func_interp_sensor_1(master_time)


df_imu_linear_z= dfn[['imu_time', 'imu_linear_acc_z']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_imu_linear_z.imu_time, df_imu_linear_z.imu_linear_acc_z)
interp_imu_linear_acc_z = func_interp_sensor_1(master_time)





### New Interpolated Dataframe

In [55]:
odor = pd.DataFrame({'master_time': master_time,
                        'odor': interp_odor,
                        'trisonica_U': interp_wind_U,
                        'trisonica_V': interp_wind_V,
                        'lat': interp_gps_lat,
                        'lon': interp_gps_lon,
                        'gps_linear_x':interp_gps_x,
                        'gps_linear_y':interp_gps_y,
                        'gps_linear_z':interp_gps_z,
                        'trisonica_pressure':interp_wind_p,
                        'trisonica_temperature':interp_wind_t,
                        'imu_angular_x': interp_imu_x,
                        'imu_angular_y': interp_imu_y,
                        'imu_angular_z': interp_imu_z,
                        'imu_linear_acc_x':interp_imu_linear_acc_x,
                        'imu_linear_acc_y':interp_imu_linear_acc_y,
                        'imu_linear_acc_z':interp_imu_linear_acc_z})

# odor.to_hdf('/home/ecc/data/DesertDataOct/Run01/Run01_Interpolated.h5',key='df', mode='w')

## Interpolating Wind and Odor

In [56]:
#ubuntu
fname='~/data/DesertDataOct/StationeryWind/20201020_172919_windy_b_data.hdf'
#mac 
# fname = '~/Documents/Myfiles/DataAnalysis/data/20201020_172919_windy_b_data.hdf'
wind_df=pd.read_hdf(fname)
wind_df.keys()

wind =pd.DataFrame
wind = wind_df.loc[(wind_df.time_epoch >= new_df.master_time.iloc[0]) & 
                   (wind_df.time_epoch <= new_df.master_time.iloc[-1])]
# df2.to_hdf('/home/ecc/data/DesertDataOct/Run01/Wind01.h5', key='df2', mode='w')

### Converting lat lon information to meters

In [57]:
from shapely.geometry import Point
odor['geometry'] = odor.apply(lambda row: Point(row.lon, row.lat), axis=1)

geodf = gpd.GeoDataFrame(odor)
geodf.crs= {'init': 'epsg:4326'}
# change the projection of geodf
geodf = geodf.to_crs("+proj=utm +zone=11 +ellps=WGS84 +datum=WGS84 +units=m +no_def")

geodf["x"] = geodf.geometry.apply(lambda row:row.x)
geodf["y"] = geodf.geometry.apply(lambda row:row.y)

#source location to meters
source = (40.79376709443614, -119.15757512132207)
u =  utm.from_latlon(40.79376709443614, -119.15757512132207)
xfromsrc= geodf.x-u[0]
geodf.insert(9,'xsrc',xfromsrc) 

yfromsrc = geodf.y-u[1]
geodf.insert(10,'ysrc', yfromsrc)

/home/ecc/.local/lib/python3.6/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


### Interpolate

In [58]:
df_odor_loc=pd.DataFrame({'odor_time': geodf.master_time,'xsrc': geodf.xsrc, 'ysrc':geodf.ysrc, 
                          'odor':geodf.odor,
                          'trisonica_U':geodf.trisonica_U, 'trisonica_V':geodf.trisonica_V,
                          'trisonica_temperature':geodf.trisonica_temperature,
                          'trisonica_pressure':geodf.trisonica_pressure,
                          'gps_linear_x':geodf.gps_linear_x,
                          'gps_linear_y':geodf.gps_linear_y,
                          'gps_linear_z':geodf.gps_linear_z,
                          'imu_angular_x': geodf.imu_angular_x,
                          'imu_angular_y': geodf.imu_angular_y,
                          'imu_angular_z': geodf.imu_angular_z,
                          'imu_linear_acc_x':geodf.imu_linear_acc_z,
                          'imu_linear_acc_y':geodf.imu_linear_acc_z,
                          'imu_linear_acc_z':geodf.imu_linear_acc_z
                         })
df_wind_loc=pd.DataFrame({'wind_time': wind.time_epoch,'U': wind.U,'V': wind.V, 'D':wind.D , 'S2':wind.S2})


df_new = pd.concat([df_odor_loc,df_wind_loc], sort=True)
combined_time = np.hstack([df_odor_loc.odor_time, df_wind_loc.wind_time])

df_new['time'] = combined_time
df_new=df_new.sort_values(by=['time'])

dt_odor = np.median(np.diff(df_new.odor_time.dropna().values))
print('dt Odor: ', dt_odor)
dt_wind = np.median(np.diff(df_new.wind_time.dropna().values))
print('dt Wind: ', dt_wind)

first_time = np.max([df_new.odor_time.min(), df_new.wind_time.min()])
last_time = np.min([df_new.odor_time.max(), df_new.wind_time.max()])
############## SELECT THE RESOLUTION HERE ####################
master_time = np.arange(first_time, last_time, dt_odor)

# create interpolation function and inteprolate sensor 1
df_odor_1= df_new[['odor_time', 'xsrc']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_odor_1.odor_time, df_odor_1.xsrc)
interp_odor_x = func_interp_sensor_1(master_time)

df_odor_2= df_new[['odor_time', 'ysrc']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_odor_2.odor_time, df_odor_2.ysrc)
interp_odor_y = func_interp_sensor_1(master_time)

df_odor_3= df_new[['odor_time', 'odor']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_odor_3.odor_time, df_odor_3.odor)
interp_odor = func_interp_sensor_1(master_time)

df_gps_x= df_new[['odor_time', 'gps_linear_x']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_gps_x.odor_time, df_gps_x.gps_linear_x)
interp_gps_x = func_interp_sensor_1(master_time)

df_gps_y= df_new[['odor_time', 'gps_linear_y']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_gps_y.odor_time, df_gps_y.gps_linear_y)
interp_gps_y = func_interp_sensor_1(master_time)

df_gps_z= df_new[['odor_time', 'gps_linear_z']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_gps_z.odor_time, df_gps_z.gps_linear_z)
interp_gps_z = func_interp_sensor_1(master_time)



df_wind_1x= df_new[['odor_time', 'trisonica_U']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_wind_1x.odor_time, df_wind_1x.trisonica_U)
interp_wind_Ux = func_interp_sensor_1(master_time)

df_wind_2x= df_new[['odor_time', 'trisonica_V']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_wind_2x.odor_time, df_wind_2x.trisonica_V)
interp_wind_Vx = func_interp_sensor_1(master_time)

df_wind_3x= df_new[['odor_time', 'trisonica_pressure']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_wind_3x.odor_time, df_wind_3x.trisonica_pressure)
interp_wind_px = func_interp_sensor_1(master_time)

df_wind_4x= df_new[['odor_time', 'trisonica_temperature']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_wind_4x.odor_time, df_wind_4x.trisonica_temperature)
interp_wind_tx = func_interp_sensor_1(master_time)

df_imu_x= df_new[['odor_time', 'imu_angular_x']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_imu_x.odor_time, df_imu_x.imu_angular_x)
interp_imu_x = func_interp_sensor_1(master_time)

df_imu_y= df_new[['odor_time', 'imu_angular_y']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_imu_y.odor_time, df_imu_y.imu_angular_y)
interp_imu_y = func_interp_sensor_1(master_time)


df_imu_z= df_new[['odor_time', 'imu_angular_z']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_imu_z.odor_time, df_imu_z.imu_angular_z)
interp_imu_z = func_interp_sensor_1(master_time)


df_imu_linear_x= df_new[['odor_time', 'imu_linear_acc_x']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_imu_linear_x.odor_time, df_imu_linear_x.imu_linear_acc_x)
interp_imu_linear_acc_x = func_interp_sensor_1(master_time)

df_imu_linear_y= df_new[['odor_time', 'imu_linear_acc_y']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_imu_linear_y.odor_time, df_imu_linear_y.imu_linear_acc_y)
interp_imu_linear_acc_y = func_interp_sensor_1(master_time)


df_imu_linear_z= df_new[['odor_time', 'imu_linear_acc_z']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_imu_linear_z.odor_time, df_imu_linear_z.imu_linear_acc_z)
interp_imu_linear_acc_z = func_interp_sensor_1(master_time)




df_wind_1= df_new[['wind_time', 'U']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_wind_1.wind_time, df_wind_1.U)
interp_wind_U = func_interp_sensor_1(master_time)

df_wind_2= df_new[['wind_time', 'V']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_wind_2.wind_time, df_wind_2.V)
interp_wind_V = func_interp_sensor_1(master_time)

df_wind_3= df_new[['wind_time', 'D']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_wind_3.wind_time, df_wind_3.D)
interp_wind_D = func_interp_sensor_1(master_time)

df_wind_4= df_new[['wind_time', 'S2']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_wind_4.wind_time, df_wind_4.S2)
interp_wind_S2 = func_interp_sensor_1(master_time)



windn = pd.DataFrame({'master_time': master_time,
                            'xsrc': interp_odor_x,
                            'ysrc': interp_odor_y,
                            'odor': interp_odor,
                            'trisonica_u': interp_wind_Ux,
                            'trisonica_v': interp_wind_Vx,
                            'trisonica_pressure':interp_wind_px,
                            'trisonica_temperature':interp_wind_tx,
                            'gps_linear_x': interp_gps_x,
                            'gps_linear_y': interp_gps_y,
                            'gps_linear_z': interp_gps_z,
                            'imu_angular_x': interp_imu_x,
                            'imu_angular_y': interp_imu_y,
                            'imu_angular_z': interp_imu_z,
                            'imu_linear_acc_x':interp_imu_linear_acc_x,
                            'imu_linear_acc_y':interp_imu_linear_acc_y,
                            'imu_linear_acc_z':interp_imu_linear_acc_z,
                      
                            'U': interp_wind_U,
                            'V': interp_wind_V,
                            'D':interp_wind_D,
                            'S2':interp_wind_S2})

windn.to_hdf('/home/ecc/data/DesertDataOct/Run0'+str(run)+'/Run0'+str(run)+'.h5',key='windn', mode='w')

dt Odor:  0.004996538162231445
dt Wind:  0.10000276565551758


In [31]:
windn

,master_time,xsrc,ysrc,odor,trisonica_u,trisonica_v,trisonica_pressure,trisonica_temperature,gps_linear_x,gps_linear_y,...,imu_angular_x,imu_angular_y,imu_angular_z,imu_linear_acc_x,imu_linear_acc_y,imu_linear_acc_z,U,V,D,S2
0,1.603220e+09,0.936368,-0.789034,0.673029,-1.298234,-0.661766,885.900024,7.394513,0.001117,0.033848,...,0.074556,-0.115579,0.007692,-10.125998,-10.125998,-10.125998,-0.590000,0.970000,148.000000,1.140000
1,1.603220e+09,0.936368,-0.789034,0.672028,-1.280317,-0.679683,885.900024,7.440302,0.002216,0.032424,...,0.078427,-0.116820,0.004062,-9.835369,-9.835369,-9.835369,-0.592997,0.968501,147.900096,1.140000
2,1.603220e+09,0.936368,-0.789034,0.670134,-1.262399,-0.697601,885.900024,7.486091,0.003315,0.031000,...,0.072145,-0.111547,0.000416,-9.349153,-9.349153,-9.349153,-0.595994,0.967003,147.800192,1.140000
3,1.603220e+09,0.936368,-0.789034,0.670388,-1.244482,-0.715518,885.900024,7.531879,0.004414,0.029577,...,0.056047,-0.100713,0.003952,-9.201790,-9.201790,-9.201790,-0.598991,0.965504,147.700288,1.140000
4,1.603220e+09,0.936368,-0.789034,0.671013,-1.226617,-0.733359,885.900024,7.577590,0.005513,0.028153,...,0.039053,-0.095657,0.000351,-9.305958,-9.305958,-9.305958,-0.601988,0.964006,147.600383,1.140000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238197,1.603221e+09,1.605504,-2.071866,0.684757,-0.414994,0.630702,885.599976,8.539708,-0.004094,0.000123,...,0.004092,0.000679,0.019499,-9.911934,-9.911934,-9.911934,-0.895580,0.509227,119.442016,1.035193
238198,1.603221e+09,1.605504,-2.071866,0.683580,-0.421039,0.621959,885.599976,8.510801,-0.004294,0.000647,...,0.004758,0.000360,0.012931,-9.911352,-9.911352,-9.911352,-0.898577,0.505231,119.142303,1.036192
238199,1.603221e+09,1.605504,-2.071866,0.683513,-0.445115,0.648041,885.599976,8.490738,-0.004494,0.001172,...,0.005282,0.000361,0.012612,-9.914024,-9.914024,-9.914024,-0.901574,0.501235,118.842591,1.037191
238200,1.603221e+09,1.605504,-2.071866,0.682266,-0.469191,0.674124,885.599976,8.470674,-0.004694,0.001696,...,0.004849,0.000309,0.014277,-9.910068,-9.910068,-9.910068,-0.904571,0.497238,118.542878,1.038190
